***Participantes (RM - NOME):***<br>
339624 - Camila<br>
339656 - Cleiton<br>
340214 - Henrique<br>
339708 - Roberto<br>
340192 - Sergio<br>

## **Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

# Bibliotecas utilizadas

In [71]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import wordcloud as wd
import re
from collections import Counter

import spacy

import nltk
from nltk.stem.rslp import RSLPStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import LinearSVC


import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.layers import Dropout, Flatten, Conv1D, Embedding
from keras.layers import Flatten
from keras.utils import plot_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pydot


In [2]:
sb.set_theme(context='notebook', style='whitegrid')

nlp = spacy.load('pt_core_news_md')
rslp = RSLPStemmer()

In [3]:
print(tf.__version__)
#print(keras.__version__)
print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0
True
Num GPUs Available:  1


# Funções externas

In [81]:
# função de Stemização completa do documento
def stemmer_text(frase):
  tokens = [rslp.stem(w) for w in frase.split()]
  return " ".join(tokens)

# função de lematização completa do documento
def lemmatizer_text(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ for w in doc]
  return " ".join(tokens)

# função de lematização somente para os verbos do documento
def lemmatizer_verbs(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ if w.pos_ == 'VERB' else w.text for w in doc]
  return " ".join(tokens)

# função de lematização apenas ADV e ADJ
def lemmatizer_adv_adj(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ for w in doc if (w.pos_ == 'ADV' or w.pos_ == 'ADJ')]
  return " ".join(tokens)  

# função para retiral o plural das palavras em português
def plural_singular(palavra):
  r2 = r"ses$|zes$|res$"
  r3 = r"ões$|ães$"
  r4 = r"ais$|éis$|óis$|uis$"
  r5 = r"is$"
  r6 = r"eis$"
  if palavra.endswith('s'):
    if re.findall(r2, palavra):
      return palavra[:-2]
    if re.findall(r3, palavra):
      return palavra[:-3] + "ão"
    if re.findall(r4, palavra):
      return palavra[:-2] + "l"
    if re.findall(r5, palavra):
      return palavra[:-1] + "l"
    if re.findall(r6, palavra):
      return palavra[:-1] + "il"
    if palavra.endswith('ns'):
      return palavra[:-2] + "m"
    return palavra[:-1]
  return palavra  

# função para executar o pré processamento
def pre(frase):
  regex = r"[`,.?:;!&\"]"
  palavras = frase.split()
  palavras = [p.lower() for p in palavras if p not in stops]
  palavras = [re.sub(regex, "", p) for p in palavras]
  palavras = [p for p in palavras if len(p) >= 3]
  palavras = [p for p in palavras if not p.isnumeric()]
  palavras = [plural_singular(p) for p in palavras]

  return " ".join(palavras)

# função para criar as bases de treino e teste vetorizadas, com Unigramas, Bigramas e Trigramas
def criar_vetores(base_treino, base_teste, coluna):
    nomes = ['CV ngram(1,1)', 'CV ngram(1,2)', 'CV ngram(1,3)', 'CV ngram(2,2)', 'CV ngram(2,3)', 'CV ngram(3,3)',
                'TF ngram(1,1)', 'TF ngram(1,2)', 'TF ngram(1,3)', 'TF ngram(2,2)', 'TF ngram(2,3)', 'TF ngram(3,3)',]
    vetores = []
    idx = 0

    for x in range(1,4):
        for y in range(1,4): 
            if (y < x):
                continue
            vect = CountVectorizer(ngram_range=(x,y), stop_words = stops, min_df=10)
            vect.fit(df_treino[coluna])
            vect_treino = vect.transform(df_treino[coluna])
            vect_teste = vect.transform(df_teste[coluna])
            vetores.append([nomes[idx], vect_treino, vect_teste])
            idx += 1

    for x in range(1,4):
        for y in range(1,4): 
            if (y < x):
                continue
            vect = TfidfVectorizer(ngram_range=(x,y), stop_words = stops, min_df=10)
            vect.fit(df_treino['texto'])
            vect_treino = vect.transform(df_treino[coluna])
            vect_teste = vect.transform(df_teste[coluna])
            vetores.append([nomes[idx], vect_treino, vect_teste])
            idx += 1

    return vetores

def testar_modelo(modelo, base_treino, base_teste, vetores, coluna):
    f1score = 0
    nome = ""
    for x in vetores:
        modelo.fit(x[1], base_treino[coluna])
        predito = modelo.predict(x[2])
        f1 = f1_score(base_teste[coluna], predito, average='weighted')
        print(x[0], f1)
        if (f1score < f1):
            f1score = f1
            nome = x[0]

    return (nome, f1score)    

In [ ]:
t = "Este fime é muito bom e especial"

d = nlp(t)
for w in d:
    print(w.text, w.pos_, w.lemma_, w.dep_)

# Carregando os Dados

In [ ]:
df = pd.read_csv('./data/reviews-pt-br.csv', encoding='utf-8')

In [ ]:
df.info()

In [ ]:
df.head(10)

## Conferindo se temos dados nulos ou duplicados

In [ ]:
df.isna().sum()

In [ ]:
df[df.duplicated()].count()

## Distribuição das respostas

In [ ]:
df.groupby('sentimento').count()

In [ ]:
# em percentual
round(df.groupby('sentimento').count().texto / df.shape[0] * 100, 2)

In [ ]:
sb.countplot(x=df.sentimento)

## Criando o conjunto de stopwords (NLTK + SPACY)

In [7]:
stopwords_nltk = nltk.corpus.stopwords.words('portuguese')
stopwords_spacy = nlp.Defaults.stop_words
stops = list(set(stopwords_spacy).union(stopwords_nltk))
print(sorted(stops))

['a', 'acerca', 'ademais', 'adeus', 'agora', 'ainda', 'algo', 'algumas', 'alguns', 'ali', 'além', 'ambas', 'ambos', 'antes', 'ao', 'aos', 'apenas', 'apoia', 'apoio', 'apontar', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'as', 'assim', 'através', 'atrás', 'até', 'aí', 'baixo', 'bastante', 'bem', 'boa', 'bom', 'breve', 'cada', 'caminho', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'com', 'como', 'comprida', 'comprido', 'conhecida', 'conhecido', 'conselho', 'contra', 'contudo', 'corrente', 'cuja', 'cujo', 'custa', 'cá', 'da', 'daquela', 'daquele', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'des', 'desde', 'dessa', 'desse', 'desta', 'deste', 'deve', 'devem', 'deverá', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'diante', 'direita', 'disso', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dá', 'dão', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'en

# Analisando o conjunto total das palavras

In [ ]:
# exemplo de críticas no nosso dataframe
print(df.texto[0])
print("-" * 40)
print(df.texto[1536])
print("-" * 40)
print(df.texto[8192])

In [ ]:
# criando um texto completo com todas as críticas
texto_completo = " ".join([texto for texto in df.texto])
texto_completo_pos = " ".join([texto for texto in df[df.sentimento == 'pos'].texto])
texto_completo_neg = " ".join([texto for texto in df[df.sentimento == 'neg'].texto])

In [ ]:
print("QTD de todas as palvaras", len(texto_completo.split()))
print("QTD de todas as palvaras das críticas positivas", len(texto_completo_pos.split()))
print("QTD de todas as palvaras das críticas negativas", len(texto_completo_neg.split()))

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo_pos)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras POSITIVAS da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo_neg)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras NEGATIVAS da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

## Criando uma função de pré processamento

In [ ]:
palavras = texto_completo.split()
len(palavras)

In [ ]:
# retirando as stopwords das palavras
palavras = [p.lower() for p in palavras if p not in stops]
len(palavras)

In [ ]:
# retirando pontuações das palavras
regex = r"[`,.?:;!&\"]"
palavras = [re.sub(regex, "", p) for p in palavras]
len(palavras)

In [ ]:
#retirando palavras menores que 3 caracteres
palavras = [p for p in palavras if len(p) >= 3]
len(palavras)

In [ ]:
# retirando os números das palavras
palavras = [p for p in palavras if not p.isnumeric()]
len(palavras)

In [ ]:
#convertendo as palavras para o singular
palavras = [plural_singular(p) for p in palavras]
len(palavras)

## Criamos uma função chamada "pre" que iremos usar em nossas análises dos modelos

# Vamos analisar quais palavras são comuns as críticas negativas e positivas

In [ ]:
todas_palavras_pos = texto_completo_pos.split()
todas_palavras_pos = [pre(w) for w in todas_palavras_pos]
todas_palavras_pos = [w for w in todas_palavras_pos if w != '']
ctpp = Counter()
ctpp.update(todas_palavras_pos)
ctpp.most_common(50)

In [ ]:
todas_palavras_neg = texto_completo_neg.split()
todas_palavras_neg = [pre(w) for w in todas_palavras_neg]
todas_palavras_neg = [w for w in todas_palavras_neg if w != '']
ctpn = Counter()
ctpn.update(todas_palavras_neg)
ctpn.most_common(50)

In [ ]:
%%time
p = ctpp.most_common()
n = ctpn.most_common()
r = [[x for x in p if x[0] == y[0]] for y in n]
r = [x for x in r if len(x) != 0]
len(r)

In [ ]:
palavras_comuns = [w[0][0] for w in r]
palavras_comuns[:100]

In [ ]:
# criando uma wordcloud das palavras filtradas
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False).fit_words(dict(ctp))

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras da Base de Dados após o Pré Pocessamento", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

## Criando as colunas tratadas

In [ ]:
%%time
#df['stem'] = df.texto.apply(stemmer_text)

In [ ]:
%time
#df['lemm'] = df.texto.apply(lemmatizer_text)

In [ ]:
%%time
#df['verb'] = df.texto.apply(lemmatizer_verbs)

In [ ]:
%%time
#df['pre'] = df.texto.apply(pre)

In [16]:
#df.to_csv("./data/criticas.csv")
df = pd.read_csv('../criticas.csv')
df = df.drop(['Unnamed: 0', 'codigo'], axis=1)

In [17]:
df.head(10)

,texto,sentimento,stem,lemm,verb,pre
0,Esse bocejo de pia de cozinha de orçamento mui...,neg,ess bocej de pia de co de orç muit baix é o ti...,Esse bocejar de pio de cozinhar de orçamentar ...,Esse bocejo de pia de cozinha de orçamento mui...,esse bocejo pia cozinha orçamento filme feito ...
1,O Bravo parece indicar que o personagem princi...,neg,"o brav parec indic que o person principal, cla...",O Bravo parecer indicar que o personagem princ...,O Bravo parecer indicar que o personagem princ...,bravo indicar personagem principal claro coraj...
2,"Durante a Guerra pela Independência do Sul, GE...",pos,"dur a guerr pel independ do sul, gener spanky ...","Durante o Guerra pelar Independência do Sul , ...","Durante a Guerra pela Independência do Sul , G...",durante guerra independência sul general spank...
3,É fora de questão que a verdadeira Anna Anders...,pos,é for de quest que a verd ann anderson não era...,É ser de questão que o verdadeiro Anna Anderso...,É fora de questão que a verdadeira Anna Anders...,verdadeira anna anderson princesa anastasia al...
4,Concordo totalmente com outro dos revisores aq...,neg,concord total com outr do revi aqu que fic sat...,Concordo totalmente com outro dos revisor aqui...,Concordo totalmente com outro dos revisores aq...,concordo totalmente outro revisor ficou satisf...
5,Obra-prima absoluta de um filme! Boa noite Mr....,pos,obra-pr absolut de um filme! boa noit mr.tom r...,Obra-prima absoluto de um filmar ! Boa noite M...,Obra-prima absoluta de um filme ! Boa noite Mr...,obra-prima absoluta filme boa noite mrtom rapi...
6,Embora a palavra megalmania seja muito usada p...,pos,emb a palavr megalman sej muit us par descrev ...,Embora o palavra megalmania ser muito usar par...,Embora a palavra megalmania seja muito usar pa...,embora palavra megalmania usada descrever gene...
7,Esta tem que ser a peça mais incrível de porca...,neg,est tem que ser a peç mais incr de porc cinema...,Esta ter que ser o pedir mais incrível de porc...,Esta tem que ser a peça mais incrível de porca...,esta peça incrível porcaria cinematográfica as...
8,Eu suponho que todas as piadas internas são o ...,neg,eu suponh que tod as pi intern são o que fez d...,Eu supor que todo o piar interno ser o que faz...,Eu supor que todas as piadas internas são o qu...,suponho piada interna munchie clássico cult pe...
9,"Se há um tema deste filme, é que as pessoas po...",pos,"se há um tem dest filme, é que as pesso pod li...","Se haver um temer dar filmar , ser que o pesso...","Se haver um tema deste filme , ser que as pess...",tema filme pessoa lidar dificuldade imaginação...


In [ ]:
stops2 = set(stops).union(palavras_comuns[:100])
len(stops2)

# Dividindo a base em treino e teste

In [18]:
# dividindo com 20% para o treino e random state = 42
df_treino, df_teste = train_test_split(
      df, 
      test_size = 0.2, 
      random_state = 42
  )

In [19]:
# distribuição das respostas do treino em %
round(df_treino.groupby('sentimento').count() / df.shape[0] * 100, 2).texto

sentimento
neg    40.07
pos    39.93
Name: texto, dtype: float64

In [20]:
# distribuição das respostas do teste em %
round(df_teste.groupby('sentimento').count() / df.shape[0] * 100, 2).texto

sentimento
neg    10.04
pos     9.96
Name: texto, dtype: float64

# Testando diversos modelos com a base total de palavras sem stopwords

## Criando os vetores com CountVectorizer e TfidfVectorizer

In [42]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'texto')

Wall time: 4min 23s


## Testando diversos modelos com os vetores criados

In [50]:
%%time
# modelo Árvore de Decisão
modelo = DecisionTreeClassifier(random_state=42, max_depth=50)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Árvore de Decisão - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.7334065607517617
CV ngram(1,2) 0.7389972124839376
CV ngram(1,3) 0.7416456410429769
CV ngram(2,2) 0.6132194031871322
CV ngram(2,3) 0.6143754350124869
CV ngram(3,3) 0.440902916358259
TF ngram(1,1) 0.7242198298707084
TF ngram(1,2) 0.7260879021556159
TF ngram(1,3) 0.7269344378693082
TF ngram(2,2) 0.611408385276224
TF ngram(2,3) 0.6152997395176724
TF ngram(3,3) 0.4438377004527155

Árvore de Decisão - CV ngram(1,3) - F1 Score: 74.16%
Wall time: 3min 53s


In [70]:
%%time
# modelo Random Forest
modelo = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Random Forest - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8570091422646389
CV ngram(1,2) 0.8580194411160453
CV ngram(1,3) 0.8610559351736565
CV ngram(2,2) 0.7738974149816571
CV ngram(2,3) 0.7747957289121654
CV ngram(3,3) 0.5939805367253124
TF ngram(1,1) 0.8527194695341529
TF ngram(1,2) 0.8548498155717084
TF ngram(1,3) 0.8569934729963394
TF ngram(2,2) 0.7777573321458281
TF ngram(2,3) 0.7803476027605121
TF ngram(3,3) 0.5922386561501604

Random Forest - CV ngram(1,3) - F1 Score: 86.11%
Wall time: 6min 19s


In [52]:
%%time
# modelo AdaBoost
modelo = AdaBoostClassifier(n_estimators=100, random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"AdaBoost - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8113072237344358
CV ngram(1,2) 0.8119893681619048
CV ngram(1,3) 0.8119893681619048
CV ngram(2,2) 0.6631463063565599
CV ngram(2,3) 0.6630078264083823
CV ngram(3,3) 0.46169500243275796
TF ngram(1,1) 0.8087066011318896
TF ngram(1,2) 0.8060494093349827
TF ngram(1,3) 0.8055787156701836
TF ngram(2,2) 0.668940166492079
TF ngram(2,3) 0.6717980213185778
TF ngram(3,3) 0.46618350077771004

AdaBoost - CV ngram(1,2) - F1 Score: 81.2%
Wall time: 3min 46s


In [53]:
%%time
# modelo Bagging Classifier
modelo = BaggingClassifier(n_estimators=20, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Bagging - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.7829529863109658
CV ngram(1,2) 0.7822625447604084
CV ngram(1,3) 0.7840829303681472
CV ngram(2,2) 0.7470878093253949
CV ngram(2,3) 0.7506578822171368
CV ngram(3,3) 0.5922569249811895
TF ngram(1,1) 0.7935963307402973
TF ngram(1,2) 0.787549924702114
TF ngram(1,3) 0.793389110907307
TF ngram(2,2) 0.749359985803243
TF ngram(2,3) 0.7512825374870906
TF ngram(3,3) 0.592533692930128

Bagging - TF ngram(1,1) - F1 Score: 79.36%
Wall time: 27min 30s


In [55]:
%%time
# modelo Extra Trees Regressor
modelo = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Extra Trees Regressor - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8708200533848273
CV ngram(1,2) 0.8759927433580862
CV ngram(1,3) 0.8758795252960768
CV ngram(2,2) 0.7811582020103887
CV ngram(2,3) 0.7781320574766459
CV ngram(3,3) 0.5925747779938603
TF ngram(1,1) 0.8709188943514428
TF ngram(1,2) 0.8750847464964969
TF ngram(1,3) 0.8756461176623055
TF ngram(2,2) 0.7963599859499786
TF ngram(2,3) 0.7935512034983379
TF ngram(3,3) 0.5923588709500518

Extra Trees Regressor - CV ngram(1,2) - F1 Score: 87.6%
Wall time: 9min 20s


In [57]:
%%time
# modelo Gradient Boosting
modelo = GradientBoostingClassifier(n_estimators=200, random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Gradient Boosting - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8131037840926045
CV ngram(1,2) 0.8154661088177287
CV ngram(1,3) 0.8136564531371888
CV ngram(2,2) 0.703771331017916
CV ngram(2,3) 0.7053132711254233
CV ngram(3,3) 0.5000896766537951
TF ngram(1,1) 0.8132784816099118
TF ngram(1,2) 0.8164773941162936
TF ngram(1,3) 0.814008556621088
TF ngram(2,2) 0.7067345728742199
TF ngram(2,3) 0.7063127642730296
TF ngram(3,3) 0.500132627555598

Gradient Boosting - TF ngram(1,2) - F1 Score: 81.65%
Wall time: 13min 26s


In [60]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
CV ngram(1,1) 0.8701556226033939
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
CV ngram(1,2) 0.8791421062562188
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
CV ngram(1,3) 0.8792543850650043
CV ngram(2,2) 0.8180140954546817
CV ngram(2,3) 0.8157729065954192
CV ngram(3,3) 0.5957558146768905
TF ngram(1,1) 0.8809187826450897
TF ngram(1,2) 0.889348947719393
TF ngram(1,3) 0.8890110656229786
TF ngram(2,2) 0.8

In [61]:
%%time
# modelo Passive Agressive
modelo = PassiveAggressiveClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Passive Agressive - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8456691927963159
CV ngram(1,2) 0.8662239768753464
CV ngram(1,3) 0.8673472012508682
CV ngram(2,2) 0.7765912918116423
CV ngram(2,3) 0.7743456179758983
CV ngram(3,3) 0.5423155547597628
TF ngram(1,1) 0.8507237969132462
TF ngram(1,2) 0.8715043635253087
TF ngram(1,3) 0.8717270286280259
TF ngram(2,2) 0.7825432505960539
TF ngram(2,3) 0.7844517846905066
TF ngram(3,3) 0.5502119859308695

Passive Agressive - TF ngram(1,3) - F1 Score: 87.17%
Wall time: 3.44 s


In [63]:
%%time
# modelo Ridge
modelo = RidgeClassifier(solver='sparse_cg', random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Ridge - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.7585067593071052
CV ngram(1,2) 0.7477234170084225
CV ngram(1,3) 0.7525534828089833
CV ngram(2,2) 0.7399582036362565
CV ngram(2,3) 0.7370462071476761
CV ngram(3,3) 0.5925886471995548
TF ngram(1,1) 0.8773200856296003
TF ngram(1,2) 0.8891242755320212
TF ngram(1,3) 0.8883345753860943
TF ngram(2,2) 0.8149814872678168
TF ngram(2,3) 0.8167834743557018
TF ngram(3,3) 0.595673890235394

Ridge - TF ngram(1,2) - F1 Score: 88.91%
Wall time: 23 s


In [64]:
%%time
# modelo SGD
modelo = SGDClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SGD - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8548745222099186
CV ngram(1,2) 0.8677979087343012
CV ngram(1,3) 0.8672275315382145
CV ngram(2,2) 0.807559686093767
CV ngram(2,3) 0.8044276726022707
CV ngram(3,3) 0.5798838824002565
TF ngram(1,1) 0.8816760564524672
TF ngram(1,2) 0.8879717331315884
TF ngram(1,3) 0.8878598659072925
TF ngram(2,2) 0.8247457475321708
TF ngram(2,3) 0.8234283422807653
TF ngram(3,3) 0.5883600913938836

SGD - TF ngram(1,2) - F1 Score: 88.8%
Wall time: 2.75 s


In [65]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8561785530659148
CV ngram(1,2) 0.8746468123072807
CV ngram(1,3) 0.8753162962608646
CV ngram(2,2) 0.8410494096777025
CV ngram(2,3) 0.8381562881984885
CV ngram(3,3) 0.5979569271742787
TF ngram(1,1) 0.8561785530659148
TF ngram(1,2) 0.8746468123072807
TF ngram(1,3) 0.8753162962608646
TF ngram(2,2) 0.8410494096777025
TF ngram(2,3) 0.8381562881984885
TF ngram(3,3) 0.5979569271742787

Naive Bayes Bernoulli - CV ngram(1,3) - F1 Score: 87.53%
Wall time: 1.66 s


In [66]:
%%time
# modelo Naive Bayes Complement
modelo = ComplementNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Complement - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8541920098423615
CV ngram(1,2) 0.8710538417137474
CV ngram(1,3) 0.8718351898036343
CV ngram(2,2) 0.8401366622752207
CV ngram(2,3) 0.8396681025173632
CV ngram(3,3) 0.5734863253834386
TF ngram(1,1) 0.8618447927514207
TF ngram(1,2) 0.8746325936041583
TF ngram(1,3) 0.8748477005896964
TF ngram(2,2) 0.8393536418952174
TF ngram(2,3) 0.8395563782718626
TF ngram(3,3) 0.5747816408564258

Naive Bayes Complement - TF ngram(1,3) - F1 Score: 87.48%
Wall time: 1.4 s


In [67]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8540793419186162
CV ngram(1,2) 0.8710538417137474
CV ngram(1,3) 0.871723063743025
CV ngram(2,2) 0.8400257852209359
CV ngram(2,3) 0.8395563782718626
CV ngram(3,3) 0.5726916420207792
TF ngram(1,1) 0.8613950353813321
TF ngram(1,2) 0.8747465261210724
TF ngram(1,3) 0.8746261871797448
TF ngram(2,2) 0.839021536888149
TF ngram(2,3) 0.8397859898224573
TF ngram(3,3) 0.5748707653722428

Naive Bayes Multinomial - TF ngram(1,2) - F1 Score: 87.47%
Wall time: 1.48 s


In [69]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
CV ngram(1,1) 0.8427503134500534
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
CV ngram(1,2) 0.8593696786901172
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
CV ngram(1,3) 0.8592543708124906
CV ngram(2,2) 0.7835525602249629
CV ngram(2,3) 0.7817598993091702
CV ngram(3,3) 0.5928683562102969
TF ngram(1,1) 0.8785747941960814
TF ngram(1,2) 0.8899243167399011
TF ngram(1,3) 0.8908224226768418
TF ngram(2,2) 0.813766579137476

# Testando os modelos com a base stematizada sem stopwords

In [94]:
%%time
# criando os vetores para Unigramas, Bigramas, Trigramas com CountVectorizer e TfidfVectorizer
vetores = criar_vetores(df_treino, df_teste, 'stem')

Wall time: 4min 35s


## apenas modelos com mais de 85% no passo anterior e tempo inferior a 5 minutos

In [95]:
%%time
# modelo Regressão Logística
modelo = LogisticRegression(solver='saga', max_iter=1000, random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Regressão Logística - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
CV ngram(1,1) 0.8706031610913865
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
CV ngram(1,2) 0.8840837095164026
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
CV ngram(1,3) 0.8848676248811861
CV ngram(2,2) 0.8376838758268568
CV ngram(2,3) 0.8376805377601975
CV ngram(3,3) 0.7150445510051537
TF ngram(1,1) 0.8225359724241443
TF ngram(1,2) 0.8231228787398793
TF ngram(1,3) 0.8235723758714172
TF ngram(2,2) 0.

In [96]:
%%time
# modelo Passive Agressive
modelo = PassiveAggressiveClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Passive Agressive - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8447715624515739
CV ngram(1,2) 0.8720642271552428
CV ngram(1,3) 0.8737499306282133
CV ngram(2,2) 0.8154524423713615
CV ngram(2,3) 0.8211824042196831
CV ngram(3,3) 0.670711521878216
TF ngram(1,1) 0.7824273705555479
TF ngram(1,2) 0.7946762938640037
TF ngram(1,3) 0.7944521435751063
TF ngram(2,2) 0.594318686498427
TF ngram(2,3) 0.5929128015931685
TF ngram(3,3) 0.35665806113545495

Passive Agressive - CV ngram(1,3) - F1 Score: 87.37%
Wall time: 4.73 s


In [97]:
%%time
# modelo Ridge
modelo = RidgeClassifier(solver='sparse_cg', random_state=42)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Ridge - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.7800723288078767
CV ngram(1,2) 0.7776023312377306
CV ngram(1,3) 0.7891719816823664
CV ngram(2,2) 0.7305320606980007
CV ngram(2,3) 0.7366007736035252
CV ngram(3,3) 0.6931774787857015
TF ngram(1,1) 0.8310286072770471
TF ngram(1,2) 0.8360933383775296
TF ngram(1,3) 0.8357547960627466
TF ngram(2,2) 0.610473749539492
TF ngram(2,3) 0.6107622720271955
TF ngram(3,3) 0.3740804219182571

Ridge - TF ngram(1,2) - F1 Score: 83.61%
Wall time: 39.1 s


In [98]:
%%time
# modelo SGD
modelo = SGDClassifier(random_state=42, n_jobs=-1)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SGD - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8593501815753932
CV ngram(1,2) 0.8739129241828761
CV ngram(1,3) 0.8732718898722627
CV ngram(2,2) 0.816242022298419
CV ngram(2,3) 0.8228637616258457
CV ngram(3,3) 0.7094704488293562
TF ngram(1,1) 0.8164689388996369
TF ngram(1,2) 0.819060757664068
TF ngram(1,3) 0.8184998012692756
TF ngram(2,2) 0.612289585227973
TF ngram(2,3) 0.6125923903751722
TF ngram(3,3) 0.373464245097864

SGD - CV ngram(1,2) - F1 Score: 87.39%
Wall time: 3 s


In [99]:
%%time
# modelo Naive Bayes Bernoulli
modelo = BernoulliNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Bernoulli - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.853279813727806
CV ngram(1,2) 0.8726032412689158
CV ngram(1,3) 0.8748100562187059
CV ngram(2,2) 0.8460315157127096
CV ngram(2,3) 0.8445608675521092
CV ngram(3,3) 0.7242011371467321
TF ngram(1,1) 0.8198725818996544
TF ngram(1,2) 0.8152285233285822
TF ngram(1,3) 0.8152326385312899
TF ngram(2,2) 0.6169621086941445
TF ngram(2,3) 0.6161082603679379
TF ngram(3,3) 0.37557593287108426

Naive Bayes Bernoulli - CV ngram(1,3) - F1 Score: 87.48%
Wall time: 1.52 s


In [100]:
%%time
# modelo Naive Bayes Complement
modelo = ComplementNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Complement - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8475371319702153
CV ngram(1,2) 0.8686958159758584
CV ngram(1,3) 0.8714971557859723
CV ngram(2,2) 0.8591156073972187
CV ngram(2,3) 0.8594120043129988
CV ngram(3,3) 0.7150760022360954
TF ngram(1,1) 0.8136591938472498
TF ngram(1,2) 0.8143208959389149
TF ngram(1,3) 0.8145369026247485
TF ngram(2,2) 0.618029123817186
TF ngram(2,3) 0.6168547481595895
TF ngram(3,3) 0.37369042341230274

Naive Bayes Complement - CV ngram(1,3) - F1 Score: 87.15%
Wall time: 1.35 s


In [101]:
%%time
# modelo Naive Bayes Multinomial
modelo = MultinomialNB()
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"Naive Bayes Multinomial - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

CV ngram(1,1) 0.8473112660817483
CV ngram(1,2) 0.868583560494826
CV ngram(1,3) 0.8714971557859723
CV ngram(2,2) 0.8590047111285145
CV ngram(2,3) 0.8595265442310434
CV ngram(3,3) 0.715056211064448
TF ngram(1,1) 0.8130790345962262
TF ngram(1,2) 0.8137398018967721
TF ngram(1,3) 0.8138439492218046
TF ngram(2,2) 0.6180576030656999
TF ngram(2,3) 0.617623906023027
TF ngram(3,3) 0.37369042341230274

Naive Bayes Multinomial - CV ngram(1,3) - F1 Score: 87.15%
Wall time: 1.33 s


In [102]:
%%time
# modelo Linear SVM
modelo = LinearSVC(random_state=42, max_iter=2000)
# calculo com todos os vetores.
f1score = testar_modelo(modelo, df_treino, df_teste, vetores, 'sentimento')

print()
print(f"SVM Linear - {f1score[0]} - F1 Score: {round(f1score[1]*100,2)}%")

C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
CV ngram(1,1) 0.8427503015472957
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
CV ngram(1,2) 0.8681338243394326
C:\Users\hgf77\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
CV ngram(1,3) 0.8676849865494912
CV ngram(2,2) 0.8082677036133574
CV ngram(2,3) 0.8142200102456199
CV ngram(3,3) 0.7003582313157055
TF ngram(1,1) 0.8324071297488379
TF ngram(1,2) 0.8368966062975588
TF ngram(1,3) 0.8364479335806516
TF ngram(2,2) 0.611141878648815

# Testando os modelos com a base lematizada sem stopwords

# MLP

In [82]:
df.head()

,texto,sentimento,stem,lemm,verb,pre
0,Esse bocejo de pia de cozinha de orçamento mui...,neg,ess bocej de pia de co de orç muit baix é o ti...,Esse bocejar de pio de cozinhar de orçamentar ...,Esse bocejo de pia de cozinha de orçamento mui...,esse bocejo pia cozinha orçamento filme feito ...
1,O Bravo parece indicar que o personagem princi...,neg,"o brav parec indic que o person principal, cla...",O Bravo parecer indicar que o personagem princ...,O Bravo parecer indicar que o personagem princ...,bravo indicar personagem principal claro coraj...
2,"Durante a Guerra pela Independência do Sul, GE...",pos,"dur a guerr pel independ do sul, gener spanky ...","Durante o Guerra pelar Independência do Sul , ...","Durante a Guerra pela Independência do Sul , G...",durante guerra independência sul general spank...
3,É fora de questão que a verdadeira Anna Anders...,pos,é for de quest que a verd ann anderson não era...,É ser de questão que o verdadeiro Anna Anderso...,É fora de questão que a verdadeira Anna Anders...,verdadeira anna anderson princesa anastasia al...
4,Concordo totalmente com outro dos revisores aq...,neg,concord total com outr do revi aqu que fic sat...,Concordo totalmente com outro dos revisor aqui...,Concordo totalmente com outro dos revisores aq...,concordo totalmente outro revisor ficou satisf...


In [83]:
df_mlp = df[['texto', 'pre']].copy()
df_mlp['target'] = [0 if x == 'neg' else 1 for x in df.sentimento]
df_mlp.head()

,texto,pre,target
0,Esse bocejo de pia de cozinha de orçamento mui...,esse bocejo pia cozinha orçamento filme feito ...,0
1,O Bravo parece indicar que o personagem princi...,bravo indicar personagem principal claro coraj...,0
2,"Durante a Guerra pela Independência do Sul, GE...",durante guerra independência sul general spank...,1
3,É fora de questão que a verdadeira Anna Anders...,verdadeira anna anderson princesa anastasia al...,1
4,Concordo totalmente com outro dos revisores aq...,concordo totalmente outro revisor ficou satisf...,0


In [84]:
# dividindo com 20% para o treino e random state = 42
df_treino_mlp, df_teste_mlp = train_test_split(
      df_mlp, 
      test_size = 0.2, 
      random_state = 42
  )
df_mlp.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44514 entries, 0 to 44513
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   44514 non-null  object
 1   pre     44514 non-null  object
 2   target  44514 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [85]:
# distribuição das respostas do treino em %
round(df_treino_mlp.groupby('target').count() / df_mlp.shape[0] * 100, 2).texto

target
0    40.07
1    39.93
Name: texto, dtype: float64

In [86]:
# distribuição das respostas do teste em %
round(df_teste_mlp.groupby('target').count() / df_mlp.shape[0] * 100, 2).texto

target
0    10.04
1     9.96
Name: texto, dtype: float64

In [ ]:
t = Tokenizer()
t.fit_on_texts(df_mlp.pre)

In [ ]:
t.document_count

In [ ]:
encoded_docs = t.texts_to_matrix(df_mlp.pre, mode='count')
encoded_docs.shape

In [91]:
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect_MLP = CountVectorizer(ngram_range=(1,2), stop_words=stops, min_df=10) 
#vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect_MLP.fit(df_treino_mlp.pre)
vect_treino_MLP = vect_MLP.transform(df_treino_mlp.pre)

In [92]:
X = vect_treino_MLP.toarray()
X.shape

(35611, 44697)

In [93]:
y = df_treino_mlp.target.values.reshape(-1,1)
y.shape

(35611, 1)

In [ ]:
#Modelo da Rede MLP
model = Sequential()
model.add(Embedding())
model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
plot_model(model)

In [ ]:
%%time
#Processo de treinamento com 80% dos dados
history  = model.fit(X, y, epochs=5, validation_split=0.2, batch_size=32)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
vect_teste = vect.transform(df_teste_mlp.pre)
X_teste = vect_teste.toarray()
X_teste.shape

In [ ]:
y_teste = df_teste_mlp.target.values.reshape(-1,1)
y_teste.shape

In [ ]:
#Processo de teste com 30% dos dados que não foram utilizados no treinamento
y_pred_classes  = model.predict_classes(X_teste)
y_pred_classes

In [ ]:
scores = model.evaluate(X_teste, y_teste)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#### **Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.